In [1]:
%pip install transformers

In [2]:
%pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 805.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=ed29dffaddc2bdd8e74ec25a244b9058908315f6175e39fb5b99da0f3ccf7af0
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [3]:
import json
import pickle
import time
import datetime
import random
import os
import csv

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torch
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from seqeval.metrics import f1_score

import matplotlib.pyplot as plt

device = torch.device("cpu")

SEED_VAL = 42

random.seed(SEED_VAL)
np.random.seed(SEED_VAL)
torch.manual_seed(SEED_VAL);  # Semicolon prevents jupyter from displaying last line as output

In [4]:
from google.colab import drive

drive.mount('/content/drive/')


Mounted at /content/drive/


In [5]:
SNIPS_PATH = "drive/MyDrive/datasets"
TRAIN_PATH = f"{SNIPS_PATH}/train_filtered.csv"
VAL_PATH = f"{SNIPS_PATH}/val_filtered.csv"
TEST_PATH = f"{SNIPS_PATH}/test_filtered.csv"
#df = pd.read_csv(TEST_PATH,sep=',')

# def load_snips_file(file_path):
#     list_pair =[]
#     with open(file_path,'r',encoding="utf8") as f:
#         for line in f:
#             split_line = line.split(',')
#             pair = split_line[1],split_line[2].strip()
#             list_pair.append(pair)
#     return list_pair


In [6]:
df_train = pd.read_csv(TRAIN_PATH, skipinitialspace=True)
df_valid = pd.read_csv(VAL_PATH, skipinitialspace=True)
df_test = pd.read_csv(TEST_PATH, skipinitialspace=True)

In [7]:
df_train['Subject'] = df_train['Subject'].apply(lambda x: None if x == 'me' else x)
df_valid['Subject'] = df_valid['Subject'].apply(lambda x: None if x == 'me' else x)
df_test['Subject'] = df_test['Subject'].apply(lambda x: None if x == 'me' else x)

df_train.dropna(inplace=True)
df_train = df_train.reset_index(drop=True)
df_valid.dropna(inplace=True)
df_valid = df_valid.reset_index(drop=True)
df_test.dropna(inplace=True)
df_test = df_test.reset_index(drop=True)

df_train

,Command,Subject,Target
0,get me the fork,fork,me
1,Bring me the soap,soap,me
2,give me the apple,apple,me
3,pass me the fork.,fork,me
4,Hand me the apple.,apple,me
...,...,...,...
377,Go to the bathroom cabinet and bring me the it...,item,melon
378,When you see the person raising their right ar...,arm,person
379,go to the bathtub and find the small dish and ...,dish,cabinet
380,"Head to the counter, locate the pickles, and b...",pickles,drawer


In [8]:
df_train = df_train.replace({r'\r|\n': ''}, regex=True)
df_valid = df_valid.replace({r'\r|\n': ''}, regex=True)
df_test = df_test.replace({r'\r|\n': ''}, regex=True)

In [9]:
# df_train.drop('id', axis=1, inplace=True)
# df_test.drop('id', axis=1, inplace=True)
# df_valid.drop('id', axis=1, inplace=True)

In [10]:
df_train.head()

,Command,Subject,Target
0,get me the fork,fork,me
1,Bring me the soap,soap,me
2,give me the apple,apple,me
3,pass me the fork.,fork,me
4,Hand me the apple.,apple,me


In [11]:
# df_train = df_train[df_train['logical_form'].str.contains('bring|put', na=False)].reset_index()
# df_test = df_test[df_test['logical_form'].str.contains('bring|put', na=False)].reset_index()
# df_valid = df_valid[df_valid['logical_form'].str.contains('bring|put', na=False)].reset_index()

## Intent ID Lookup

In order to input the intent labels into our model we will need to **Create a dictionary mapping each intent name to an integer ID (starting with 0), and assign the dictionary to intent_labeltoid**.

In [12]:
#ograniczamy się do bring i put

intents = df_train['Subject'].unique().tolist()
intents_valid = df_valid['Subject'].unique().tolist()
intents_test = df_test['Subject'].unique().tolist()

intents = [*intents, *intents_valid, *intents_test]

#usuwanie duplikatów
intents = list(set(intents))

intent_labeltoid = {intents[i]: i  for i in range(len(intents))}
intent_labeltoid

{'coke': 0,
 'apple': 1,
 'table': 2,
 'fish': 3,
 'chips': 4,
 'objects': 5,
 'coca': 6,
 'fridge': 7,
 'milk': 8,
 'melon': 9,
 'napkin': 10,
 'it': 11,
 'shampoo': 12,
 'bag': 13,
 'spaghetti': 14,
 'pear': 15,
 'bown': 16,
 'BAG': 17,
 'chocolate': 18,
 'things': 19,
 'thing': 20,
 'fro': 21,
 'toothpaste': 22,
 'tray': 23,
 'glass': 24,
 'tuna': 25,
 'object': 26,
 'pickle': 27,
 'water': 28,
 'towel': 29,
 'dresser': 30,
 'beer': 31,
 'soap': 32,
 'dish': 33,
 'stuff': 34,
 'peach': 35,
 'sponge': 36,
 'pringles': 37,
 'items': 38,
 'spoon': 39,
 'machine': 40,
 'teaspoon': 41,
 'most': 42,
 'privilege': 43,
 'arm': 44,
 'flakes': 45,
 'person': 46,
 'Robin': 47,
 'pasta': 48,
 'Pringles': 49,
 'pickles': 50,
 'knife': 51,
 'them': 52,
 'dishes': 53,
 'cereal': 54,
 'Alex': 55,
 'bowl': 56,
 'box': 57,
 'cookies': 58,
 'cabinet': 59,
 'mug': 60,
 'way': 61,
 'tea': 62,
 'article': 63,
 'p': 64,
 'fork': 65,
 'banana': 66,
 'Jordan': 67,
 'robot': 68,
 'item': 69}

In [13]:
df_train_list = df_train.values.tolist()
df_test_list = df_test.values.tolist()
df_valid_list = df_valid.values.tolist()
for lista in [df_train_list, df_test_list, df_valid_list]:
    for item in lista:
        item.reverse()
        #do trenowania subject
        item.pop(0)
        #do trenowania target
        # item.pop(1)
df_valid_list

[['soap', 'bring me the soap'],
 ['apple', 'Bring me the apple.'],
 ['pear', 'bring the pear to the sofa'],
 ['coke', 'Put a coke on the high chair.'],
 ['dish', 'Put the big dish on the shower.'],
 ['apple', 'Move the apple to the side table'],
 ['dishes', 'get the large dishes from the sink'],
 ['shampoo', 'Bring me the shampoo from the sink.'],
 ['fork', 'Take the fork over to the microwave.'],
 ['bag', 'take the bag from the cabinet to the bar'],
 ['cereal', 'Move the cereal to the bathroom cabiinet'],
 ['objects', 'Bring me the lightest objects on the sofa.'],
 ['knife', 'relocate knife from bed to cutlery drawer.'],
 ['object', 'Bring me the rightmost object on the stove'],
 ['melon', 'Please take that melon to that coffee  table'],
 ['thing', 'bring me the biggest thing from the bathroom'],
 ['fish', 'I need you to retrieve the tuna fish for me.'],
 ['Robin', 'Find Robin near the tv and give her the sponge.'],
 ['melon', 'take this melon to the table in the living room'],
 ['obj

In [14]:
#How many training examples are there for each intent?
df_train['Subject'].value_counts()

Subject
object       60
item         36
objects      23
thing        19
flakes       11
             ..
pickle        1
bown          1
privilege     1
Alex          1
tea           1
Name: count, Length: 61, dtype: int64

In [15]:
# def create_mini_training_set(examples_per_intent):
#     intent_array = np.array(df_train_list)[:,0]
#     mini_batch =[]
#     for intent in intents:
#         add = intent_array[intent_array==intent]
#         shuffled_indicies=np.random.RandomState(seed=42).permutation(len(add))
#         class_indicies=shuffled_indicies[:examples_per_intent]
#         sampled_set = np.array(df_train_list)[class_indicies]
#         mini_batch.append(sampled_set)
#     mini_batch = np.array(mini_batch)
#     mini_set = mini_batch.transpose(1,0,2).reshape(-1,mini_batch.shape[2])
#     return mini_set

In [16]:
import re

def get_pad_length():
    all_train_examples_sentences = df_train['Command']
    word_length = []
    for sentence in all_train_examples_sentences:
        number_words = len(re.findall(r'\b\w+\b|<\w+>', sentence))
        word_length.append(number_words)
    return max(word_length)

PAD_LEN = get_pad_length()

In [17]:
PAD_LEN

20

In [18]:
INTENT_DIM = 123

## BERT Tokenizer


In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

test_utterance = "give <object> to me"

print(tokenizer.encode_plus(
            test_utterance, add_special_tokens=True, max_length=PAD_LEN, pad_to_max_length=True,
            truncation=True, return_attention_mask=True, return_tensors='pt'
    ))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

{'input_ids': tensor([[ 101, 2507, 1026, 4874, 1028, 2000, 2033,  102,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [20]:
def examples_to_dataset(examples):
    input_ids = []
    attention_masks = []
    labels = []
    for instance in examples:
        token_dict = tokenizer.encode_plus(
                instance[1], add_special_tokens=True, max_length=PAD_LEN, pad_to_max_length=True,
                truncation=True, return_attention_mask=True, return_tensors='pt')
        input_ids.append(token_dict['input_ids'])
        attention_masks.append(token_dict['attention_mask'])
        labels.append(torch.tensor(intent_labeltoid[instance[0]]).type(torch.LongTensor))

    input_ids = torch.cat(input_ids)
    attention_masks = torch.cat(attention_masks)
    labels = torch.stack(labels)


    dataset = TensorDataset(input_ids, attention_masks, labels)

    return dataset

In [21]:
#prepare the validation/test dataloaders
val_dataset = examples_to_dataset(df_valid_list)
test_dataset = examples_to_dataset(df_test_list)
BATCH_SIZE = 50
validation_dataloader = DataLoader(val_dataset, sampler=RandomSampler(df_valid_list), batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(df_test_list), batch_size=BATCH_SIZE)

# Training Setup

BERT will output a matrix tensor of shape batch size by #intents, which consists of the intent probabilities for each utterance in the batch. In order to compute the accuracy of the predictions, this prediction matrix is compared with the tensor of correct label IDs (a tensor of size #intents).



**The method `accuracy()`, which takes in the predictions (shape batch size by #intents) as a numpy array, and the correct labels as a numpy array (shape #intents), and returns the floating point accuracy of those predictions in the range \[0,1\]**

In [22]:
def get_accuracy(preds, labels):
    pred_convd = np.argmax(preds,1).flatten()
    labels_flat = labels.flatten()
    correct_labels = np.equal(pred_convd,labels_flat).sum()
    accuracy_value = correct_labels/len(labels)
    return accuracy_value

In [23]:
# Quick tests for the implementation of accuracy.

preds1 = np.array([[1,2,3], [1,3,2], [3,2,1]])

assert get_accuracy(preds1, np.array([2,1,0])) == 1.0
assert get_accuracy(preds1, np.array([2,2,0])) == 2/3
assert get_accuracy(preds1, np.array([3,2,1])) == 0.0

In [24]:
count=0
for batch in tqdm(list(test_dataloader)):
    a,b,c= batch[0],batch[1],batch[2]
    count+=1
    if count ==1:
        break
print(a);print(b);print(c)
print(batch)
print(len(list(test_dataloader)))

  0%|          | 0/3 [00:00<?, ?it/s]

tensor([[  101,  2131,  2033,  1996,  5442,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  3288,  2033,  1996,  4524,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  2507,  2033,  1996, 25742,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  3531,  3413,  2033,  1996, 14757,  1012,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  2404,  1996, 24857,  2006,  1996,  2793,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  3288,  2033,  2019,  6207,  3531,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  3413,  2745,  1037,  5572, 15

In [25]:
def evaluate(model, dataloader):
    model.eval()

    accuracy=[]

    for batch in tqdm(list(dataloader)):
        b_input_ids,b_input_mask,b_labels = batch


        with torch.no_grad():
            (loss, logits) = model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels, return_dict=False)

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        #logit_probability =torch.nn.Softmax(logits)
        batch_accuracy = get_accuracy(logits, label_ids)
        accuracy.append(batch_accuracy)
    avg_accuracy = np.mean(accuracy)  # TODO Compute final accuracy
    print("Validation Accuracy: {}".format(avg_accuracy))
    return avg_accuracy


- The input IDs, input mask, and labels are obtained from the dataloader. These inputs are passed through the model to get a prediction. After which the loss is computed for each batch.
- Thus the batch losses over time is monitored to compute the average training loss for each epoch. After every 5 batches,the validation accuracy is computed.

In [26]:

def train(model, dataloader, epochs):
    optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    for epoch_i in range(0, EPOCHS):
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))

        model.train()
        #n_iteration = 0
        accuracy = []
        total_train_loss = []

        for step, batch in tqdm(list(enumerate(train_dataloader))):
            # get input IDs, input mask, and labels from batch
            b_input_ids,b_input_mask,b_labels = batch

            model.zero_grad()
            #pass inputs through model
            (loss, logits) = model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels, return_dict=False)
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            # Add to total_train_loss
            total_train_loss.append(loss)
            #logit_probability =torch.nn.Softmax(logits)
            batch_accuracy = get_accuracy(logits, label_ids)
            accuracy.append(batch_accuracy)
            #n_iteration += 1
        # Compute average train loss
        new_loss = [x.cpu().detach().numpy() for x in total_train_loss]
        avg_train_loss = np.mean(new_loss)
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Average Training accuracy: {0:.2f}".format(np.mean(accuracy)))
    #validation_accuracy =evaluate(bert_model, validation_dataloader)



In [28]:
BATCH_SIZE = 16

EPOCHS = 10


train_dataset = examples_to_dataset(df_train_list)
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=BATCH_SIZE)


bert_model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = INTENT_DIM,
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

train(bert_model, train_dataloader, EPOCHS)

print("Evaluating on test set:")
print("Test accuracy:", evaluate(bert_model, test_dataloader))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


======== Epoch 1 / 10 ========


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/24 [00:00<?, ?it/s]

  Average training loss: 4.67
  Average Training accuracy: 0.07
======== Epoch 2 / 10 ========


...

  Average training loss: 2.84
  Average Training accuracy: 0.56
======== Epoch 10 / 10 ========


  0%|          | 0/24 [00:00<?, ?it/s]

  Average training loss: 2.80
  Average Training accuracy: 0.56
Evaluating on test set:


  0%|          | 0/3 [00:00<?, ?it/s]

Validation Accuracy: 0.5628571428571428
Test accuracy: 0.5628571428571428
